# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [2]:
def movie_title_url_actor():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url, headers)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    movie_names = []
    main_actors = []
    
    # 각 영화가 존재하는 테이블에 대한 for문, 5개로 제한
    for i, tags in enumerate(soup.find_all(class_ ='lst_dsc', limit=5)):
        
        # 영화 제목 크롤링
        movie_name = []
        movie_tag = tags.find(class_='tit').find('a') # 영화제목이 담긴 태그
        movie_name.append(movie_tag.text)
        
        # 각 영화의 주연배우 3인 크롤링
        main_actor = []
        actor_tags = tags.find(class_='info_txt1').find('dd').find_next('dd').find_next('dd') # 배우 담긴 태그
        for j, tag in enumerate(actor_tags.find_all('a', limit=3)):
            main_actor.append(tag.text)
            
        movie_names.append(movie_name)
        main_actors.append(main_actor)
    
    return movie_names, main_actors

### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [3]:
def get_grade():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url, headers)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 각 영화에 대한 관람객, 기자/평론가, 네티즌 평점 리스트
    aud_scores = []
    critic_scores = []
    netizen_scores = []
    
    # 평점 데이터 크롤링
    for i, tags in enumerate(soup.find_all(class_ ='lst_dsc', limit=5)):
        
        # 정규식을 이용한 영화 코드 번호 찾기
        code = re.compile('code=[0-9]+')
        sub_url = code.findall(tags.find(class_='tit').find('a').get('href'))
        
        # 평점 확인 위한 url
        movie_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?' + sub_url[0]
        res = requests.get(movie_url)
        html = res.text
        score_table = BeautifulSoup(html, 'html.parser')
        
        # 평점이 있는 table 찾기
        score_tags = score_table.find(class_="main_score")
        for i, tag in enumerate(score_tags.find_all(class_="star_score", limit=3)):
            score = float(tag.findAll('em')[0].text + tag.findAll('em')[1].text + tag.findAll('em')[2].text + tag.findAll('em')[3].text)
            if i == 0:
                aud_scores.append(score)
            elif i == 1:
                critic_scores.append(score)
            else:
                netizen_scores.append(score)
            
    return aud_scores, critic_scores, netizen_scores

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [4]:
def get_reviews():
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res = requests.get(url, headers)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 평점, 닉네임, 리뷰 본문 리스트
    movie_scores = []
    movie_nicknames = []
    movie_reviews = []
    
    # 리뷰 데이터 크롤링
    for i, tags in enumerate(soup.find_all(class_ ='lst_dsc', limit=5)):
        
        # 정규식을 이용한 영화 코드 번호 찾기
        code = re.compile('code=[0-9]+')
        sub_url = code.findall(tags.find(class_='tit').find('a').get('href'))
        
        # 관람객 평점 및 리뷰 확인 위한 url
        # 10건 우선 가져오기
        aud_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?' + sub_url[0]
        res = requests.get(aud_url)
        html = res.text
        aud_table = BeautifulSoup(html, 'html.parser')
        aud_tags = aud_table.find(class_='score_result')
        
        # 각 영화에 대한 평점, 닉네임, 리뷰 리스트
        scores = []
        nicknames = []
        reviews = []
        
        for i, tag in enumerate(aud_tags.find_all('li')):
            # 평점
            score = tag.find(class_='star_score').find('em').text
            scores.append(score)
            
            # 평점 작성자 닉네임
            nickname = tag.find('dl').find('span').text
            nicknames.append(nickname)
            
            # 리뷰 본문
            review = tag.find(class_='score_reple').find(id = '_filtered_ment_' + str(i)).text.strip()
            reviews.append(review)
            
        # 다음 10건 가져오기
        aud_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?' + sub_url[0] + '&page=2'
        res = requests.get(aud_url)
        html = res.text
        aud_table = BeautifulSoup(html, 'html.parser')
        aud_tags = aud_table.find(class_='score_result')
        
        for i, tag in enumerate(aud_tags.find_all('li')):
            # 평점
            score = tag.find(class_='star_score').find('em').text
            scores.append(score)
            
            # 평점 작성자 닉네임
            nickname = tag.find('dl').find('span').text
            nicknames.append(nickname)
            
            # 리뷰 본문
            review = tag.find(class_='score_reple').find(id = '_filtered_ment_' + str(i)).text.strip()
            reviews.append(review)
        
        movie_scores.append(scores)
        movie_nicknames.append(nicknames)
        movie_reviews.append(reviews)
        
    return movie_scores, movie_nicknames, movie_reviews

### 4. 저장하기

In [5]:
def save():
    data = pd.DataFrame()
    
    # 2차원 리스트를 1차원 리스트 형태로 만들어 저장
    data['movie_name'] = sum(movie_title_url_actor()[0], []) # 리스트 형태 지우기 위해 적용
    data['main_actors'] = movie_title_url_actor()[1]
    
    # 모두 1차원 리스트이므로 그대로 저장
    data['aud_score'], data['critic_score'], data['netizen_score'] = get_grade()
    
    # 2차원 리스트 형태로 저장됨
    data['scores'], data['nicknames'], data['reviews'] = get_reviews()
    
    # 각 영화의 20개의 리뷰를 long form으로 만들기 위해 리뷰가 아닌 데이터 중복해서 쌓음
    data = pd.DataFrame([[i[1], i[2], i[3], i[4], i[5], i[6][j], i[7][j], i[8][j]] for i in data.itertuples() for j in range(20)]
            , columns = ['movie_name', 'main_actors', 'aud_score', 'critic_score', 'netizen_score', 'score', 'nickname', 'reviews'])
    
    # csv로 저장
    data.to_csv('naver_movie_top5.csv', encoding='cp949')
    
    return data

### 5. 크롤링하기

In [6]:
data = save()

In [7]:
display(data)

,movie_name,main_actors,aud_score,critic_score,netizen_score,score,nickname,reviews
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.87,1,어쩌라고(dpfk****),아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
1,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.87,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
2,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.87,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
3,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.87,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
4,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.87,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
...,...,...,...,...,...,...,...,...
95,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,1,지애(maln****),최근에 본 것 중에 제일 최악..
96,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,삡(jiny****),김남길 눈빛이 다했다..재밌어요
97,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,maimai(enma****),진짜 심장 쫄깃하고 무서움 ㄷㄷㄷ 이건 극장에서 봐야함
98,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,난꽃이다(play****),무서운거 진짜 못보는대 혼자 봤거든요 걱정하시는분들 그걸 감안하고라도 보시는거 추천...
